In [5]:
def get_codons(seq: str, frame: int):
    codons = []
    for i in range(frame, len(seq) - 2, 3):
        codons.append(seq[i:i+3])
    return codons


In [7]:
start_codon = "ATG"
stop_codons = ["TAA", "TAG", "TGA"]

def find_pairs(codons: list[str]):
    pairs = []
    start_from = 0
    while True:
        try:
            start_index = codons.index(start_codon, start_from)
        except:
            return pairs
        stop_index = len(codons)
        for stop_codon in stop_codons:
            try:
                i = codons.index(stop_codon, start_index + 1)
                stop_index = min(stop_index, i)
            except:
                continue
        if stop_index != len(codons):
            pairs.append(codons[start_index:stop_index+1])
        start_from = start_index + 1

In [8]:
from Bio import SeqIO
from Bio.Seq import Seq

files = [
    'viruses/data/bacterial1.fasta',
    'viruses/data/bacterial2.fasta',
    'viruses/data/bacterial3.fasta',
    'viruses/data/bacterial4.fasta',
    'viruses/data/mamalian1.fasta',
    'viruses/data/mamalian2.fasta',
    'viruses/data/mamalian3.fasta',
    'viruses/data/mamalian4.fasta',
]
records = []
for file in files:
    for record in SeqIO.parse(file, 'fasta'):
        seq = str(record.seq)
        reverse_seq = str(record.seq.reverse_complement())
        codon_sequences = [
            get_codons(seq, 0),
            get_codons(seq, 1),
            get_codons(seq, 2),
            get_codons(reverse_seq, 0),
            get_codons(reverse_seq, 1),
            get_codons(reverse_seq, 2)
        ]
        valid_sequences = []
        for codon_sequence in codon_sequences:
            codon_pairs = find_pairs(codon_sequence)
            for start_stop_seq in codon_pairs:
                if (len(start_stop_seq) > 33): # paimam tik 100+ bp
                    valid_sequences.append(str(Seq(''.join(start_stop_seq)).translate()))
        records.append({"name": record.name, "proteins": valid_sequences})

records

[{'name': 'Lactococcus_phage',
  'proteins': ['MQTQNGGRPTILPKMYEEPLFSQIIDKIESGCNDREIYTSLHCSAKTFRKWRDDNIKAYDEAKSIARGNLLELAESALASKLTVRTLKETETIYDANGNVEKVKVKEKELDKDSLVAMMVAKAGNPELYNPTEWRRLQQEESSANDLKAKIEELDDYKLSKYETPKIEVPKGFE*',
   'MYEEPLFSQIIDKIESGCNDREIYTSLHCSAKTFRKWRDDNIKAYDEAKSIARGNLLELAESALASKLTVRTLKETETIYDANGNVEKVKVKEKELDKDSLVAMMVAKAGNPELYNPTEWRRLQQEESSANDLKAKIEELDDYKLSKYETPKIEVPKGFE*',
   'MMVAKAGNPELYNPTEWRRLQQEESSANDLKAKIEELDDYKLSKYETPKIEVPKGFE*',
   'MVAKAGNPELYNPTEWRRLQQEESSANDLKAKIEELDDYKLSKYETPKIEVPKGFE*',
   'MYYLNKMLEYNKENGIIINKYIRKTIQKQIRIHNKYIYRYDRVTQAIEWIEDNFYLTTGNLMKIKLHPVQKWWYELMLGYDMVDEKGIQVNLVNEIFLNLGRGSGKSSLMATRVLNWMILGGQYGGESLVIAYDNTQARHVFDQVRNQTEASDTLRVYNENKIFKSTKQGLEFTAFKTTFKKQTNDTLRAQGGNSSLNIFDEVHTYGEDITESVNKGSRQKQDNWQSIYITSGGLKRDGLYDKLVERFKSEEEFYNDRSFGLLYMLENHEQVKDKKNWTMALPLIGNVPKWSGVVEEYELAQGDPALQNKFLAFNMGLPMQDTAYYFTPQDTKLTDFNLSVFNKNRTYVGIDLSLIGDLTAVSFVCELEGKTYSHTLTFSVRSQYEQLDTEQQELWTEFVDRGELILLDTEYINVNDLIPYINDFRTKTGCRLRKIGYDPARYEILKGLIERYFFDKDGDNQRAIRQGFSMNDY

In [13]:
import numpy as np
from scipy.spatial.distance import euclidean

amino_acids = list("ACDEFGHIKLMNPQRSTVWY")
all_dicodons = [aa1 + aa2 for aa1 in amino_acids for aa2 in amino_acids]

def calculate_codon_frequencies(proteins):
    codon_freq = {aa: 0 for aa in amino_acids}
    total_codons = 0

    for protein in proteins:
        for aa in protein:
            if aa == '*':  # Drop stop codons
                continue
            total_codons += 1
            codon_freq[aa] += 1

    if total_codons > 0:
        codon_freq = {aa: count / total_codons for aa, count in codon_freq.items()}
    return codon_freq

def calculate_dicodon_frequencies(proteins):
    dicodon_freq = {dicodon: 0 for dicodon in all_dicodons}
    total_dicodons = 0

    for protein in proteins:
        for i in range(len(protein) - 1):
            dicodon = protein[i:i+2]
            if '*' in dicodon:  # Drop dicodons with stop codons
                continue
            total_dicodons += 1
            dicodon_freq[dicodon] += 1

    if total_dicodons > 0:
        dicodon_freq = {dicodon: count / total_dicodons for dicodon, count in dicodon_freq.items()}
    return dicodon_freq

num_records = len(records)

def calculate_distance(frequencies1, frequencies2):
    all_keys = set(frequencies1.keys()).union(frequencies2.keys())
    vec1 = np.array([frequencies1.get(k, 0.0) for k in all_keys])
    vec2 = np.array([frequencies2.get(k, 0.0) for k in all_keys])
    
    return euclidean(vec1, vec2)

def gen_matrix(compare_by):
    matrix = np.zeros((num_records, num_records))

    for i, record1 in enumerate(records):
        codon_freq1 = calculate_codon_frequencies(record1['proteins'])
        dicodon_freq1 = calculate_dicodon_frequencies(record1['proteins'])
        for j, record2 in enumerate(records):
            if i < j:
                codon_freq2 = calculate_codon_frequencies(record2['proteins'])
                dicodon_freq2 = calculate_dicodon_frequencies(record2['proteins'])

                codon_distance = calculate_distance(codon_freq1, codon_freq2)
                dicodon_distance = calculate_distance(dicodon_freq1, dicodon_freq2)
                
                total_distance = codon_distance if compare_by == 'codon' else dicodon_distance

                matrix[i][j] = matrix[j][i] = total_distance

    print(f'By {"Codon freq" if compare_by == "codon" else "Dicodon freq"}')
    print(num_records)
    for i, record in enumerate(records):
        distances = " ".join([f"{d:.3f}" for d in matrix[i]])
        print(f"{record['name']} {distances}")

gen_matrix('codon')
print()
gen_matrix('dicodon')

By Codon freq
8
Lactococcus_phage 0.000 0.051 0.044 0.027 0.061 0.085 0.083 0.102
KM389305.1 0.051 0.000 0.061 0.044 0.055 0.045 0.071 0.069
NC_028697.1 0.044 0.061 0.000 0.052 0.065 0.080 0.107 0.089
KC821626.1 0.027 0.044 0.052 0.000 0.058 0.079 0.072 0.100
coronavirus 0.061 0.055 0.065 0.058 0.000 0.064 0.083 0.081
adenovirus 0.085 0.045 0.080 0.079 0.064 0.000 0.101 0.036
U18337.1 0.083 0.071 0.107 0.072 0.083 0.101 0.000 0.123
herpesvirus 0.102 0.069 0.089 0.100 0.081 0.036 0.123 0.000

By Dicodon freq
8
Lactococcus_phage 0.000 0.028 0.028 0.025 0.032 0.037 0.037 0.044
KM389305.1 0.028 0.000 0.029 0.024 0.026 0.022 0.030 0.032
NC_028697.1 0.028 0.029 0.000 0.028 0.032 0.035 0.044 0.040
KC821626.1 0.025 0.024 0.028 0.000 0.028 0.033 0.032 0.041
coronavirus 0.032 0.026 0.032 0.028 0.000 0.028 0.034 0.037
adenovirus 0.037 0.022 0.035 0.033 0.028 0.000 0.039 0.024
U18337.1 0.037 0.030 0.044 0.032 0.034 0.039 0.000 0.048
herpesvirus 0.044 0.032 0.040 0.041 0.037 0.024 0.048 0.000
